In [1]:
import pandas as pd
import numpy as np
import re
import textract

In [16]:
import tabula

In [2]:
#extraigo texto
archivo= 'tresalia\Bancos\Banamex_nacional\pdf\Banamex Inv a la Vista.pdf'

In [3]:
text = textract.process(archivo)

In [4]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [5]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [6]:
pdf

['ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 'Registro Federal de Contribuyentes: Página:',
 '99999999',
 'XXX999999XX9 1 de 7',
 '0005607',
 'Suc. 165 C.F. POLANCO,CDMX PRESIDENTE MAZARIK 375 POLANCO IV SECCION',
 'INVERSION EMPRESARIAL',
 'GAT Nominal antes de impuestos',
 '2.75%',
 'GAT Real antes de impuestos',
 '-0.80%',
 'Interés Aplicable o Rendimientos',
 '$510.26',
 'Comisiones efectivamente cobradas',
 'No Aplica',
 'La GAT real es el rendimiento que obtendría',
 'después de descontar la inflación estimada',
 'XCDOXIRXLEOXCNXCIXIAÓXNXXXXXXXXXXXX',
 'OT',
 '00000 DELEGACIÓN, CIUDAD DE MEXICO C.R.11801',
 'La fecha de corte es la indicada después de la leyenda "ESTADO DE CUENTA AL". Salvo que expresamente se determine otra moneda, todas las cifras contenidas',
 'en el estado de cuenta se encuentran denominadas en Pesos Moneda Nacional.',
 'Su estado de cuenta contiene',
 'PRODUCTO/SERVICIO INVERSION EMPRESARIAL',
 'información de los siguientes productos',
 'R

In [7]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020
1,CLIENTE:
2,Registro Federal de Contribuyentes: Página:
3,99999999
4,XXX999999XX9 1 de 7


In [8]:
#periodo
periodo_lista=[]

for i in pdf:
    if i.startswith('PERIODO') or i.startswith('Periodo') or i.startswith('periodo'):
        periodo_lista.append(i)
        


In [9]:
periodo=periodo_lista[0]
periodo

'PERIODO DEL 01/SEP/2020 AL 30/SEP/2020'

In [10]:
#numero de cliente
cliente_lista=[]
for i in pdf:
    if i.startswith('CLIENTE') or i.startswith('Cliente') or i.startswith('cliente'):
        cliente_lista.append(i)
cliente_lista 

['CLIENTE:',
 'CLIENTE:',
 'CLIENTE: Página:',
 'CLIENTE: Página:',
 'CLIENTE: 99999999']

In [11]:
cliente= cliente_lista[-1]
cliente

'CLIENTE: 99999999'

In [12]:
#direecion
direccion_lista=[]
for i in pdf:
    if re.search('DIRECCIÓN', i):
        direccion_lista.append(i)
direccion_lista
direccion=direccion_lista[0]
direccion

'\x0cXXXXXXXX DIRECCIÓN COLONIA 00000 DELEGACIÓN, CIUDAD DE MEXICO C.R.11801'

In [13]:
pdf

['ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 'Registro Federal de Contribuyentes: Página:',
 '99999999',
 'XXX999999XX9 1 de 7',
 '0005607',
 'Suc. 165 C.F. POLANCO,CDMX PRESIDENTE MAZARIK 375 POLANCO IV SECCION',
 'INVERSION EMPRESARIAL',
 'GAT Nominal antes de impuestos',
 '2.75%',
 'GAT Real antes de impuestos',
 '-0.80%',
 'Interés Aplicable o Rendimientos',
 '$510.26',
 'Comisiones efectivamente cobradas',
 'No Aplica',
 'La GAT real es el rendimiento que obtendría',
 'después de descontar la inflación estimada',
 'XCDOXIRXLEOXCNXCIXIAÓXNXXXXXXXXXXXX',
 'OT',
 '00000 DELEGACIÓN, CIUDAD DE MEXICO C.R.11801',
 'La fecha de corte es la indicada después de la leyenda "ESTADO DE CUENTA AL". Salvo que expresamente se determine otra moneda, todas las cifras contenidas',
 'en el estado de cuenta se encuentran denominadas en Pesos Moneda Nacional.',
 'Su estado de cuenta contiene',
 'PRODUCTO/SERVICIO INVERSION EMPRESARIAL',
 'información de los siguientes productos',
 'R

In [14]:
#localizo la tabla a partir de la palabra "detalle operaciones"
indices_detalle=[]
for i in range(len(pdf)):
    if re.search('detalle de operaciones',pdf[i].lower()):
        indices_detalle.append(i)

In [15]:
indices_detalle

[47, 65]

In [20]:
tabla=tabula.read_pdf(archivo, pages='all')

# pagina 1

In [29]:
tabla_pag_1= pdf[indices_detalle[0]: indices_detalle[1]]
tabla_pag_1

['DETALLE DE OPERACIONES',
 'FECHA',
 '01 SEP 01 SEP 02 SEP 03 SEP 03 SEP',
 'CONCEPTO',
 'SALDO ANTERIOR INTERESES AL 1.27% INTERESES AL 1.27% RETIRO 72825 INTERESES AL 1.23%',
 'RETIROS 8,000.00',
 'DEPOSITOS',
 '209.13 209.13',
 '186.34',
 'SALDO 10,366.75 10,575.88 10,785.01',
 '2,971.35',
 '000182.B10OTDA006.OD.0930.01',
 '\x0cTRESALIA CAPITAL SA DE CV',
 'ESTADO DE CUENTA AL 30 DE SEPTIEMBRE DE 2020',
 'CLIENTE:',
 '62538775',
 'Página:',
 '2 de 7']

In [31]:
#fechas
fecha_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('fecha',tabla_pag_1[i].lower()):
        fecha_lista_pag_1.append(tabla_pag_1[i+1])
        
fecha_lista_pag_1

['01 SEP 01 SEP 02 SEP 03 SEP 03 SEP']

In [47]:
#separo
fechas_pag_1=re.findall(r'\d{2} \w{3}',fecha_lista_pag_1[0])

In [49]:
print(fechas_pag_1)
len(fechas_pag_1) #si son 5 debe de haber 5 valores para cada columna por la pagina 1

['01 SEP', '01 SEP', '02 SEP', '03 SEP', '03 SEP']


5

In [33]:
#conceptos
conceptos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('concepto',tabla_pag_1[i].lower()):
        conceptos_lista_pag_1.append(tabla_pag_1[i+1])
        
conceptos_lista_pag_1

['SALDO ANTERIOR INTERESES AL 1.27% INTERESES AL 1.27% RETIRO 72825 INTERESES AL 1.23%']

In [42]:
conceptos_prueba1=re.split('SALDO | INTERESES | RETIRO ',conceptos_lista_pag_1[0])
conceptos_prueba1

['', 'ANTERIOR', 'AL 1.27%', 'AL 1.27%', '72825', 'AL 1.23%']

In [45]:
conceptos_pag_1=[]
for i in range(len(conceptos_prueba1)):
    if conceptos_prueba1[i].startswith('ANTERIOR'):
        j='SALDO '+conceptos_prueba1[i]
        conceptos_pag_1.append(j)
    elif conceptos_prueba1[i].startswith('AL'):
        j='INTERES '+conceptos_prueba1[i]
        conceptos_pag_1.append(j)
    elif conceptos_prueba1[i].isdigit():
        j='RETIRO '+ conceptos_prueba1[i]
        conceptos_pag_1.append(j)

conceptos_pag_1

['SALDO ANTERIOR',
 'INTERES AL 1.27%',
 'INTERES AL 1.27%',
 'RETIRO 72825',
 'INTERES AL 1.23%']

In [52]:
#retiros
retiros_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('retiros',tabla_pag_1[i].lower()):
        retiros_lista_pag_1.append(tabla_pag_1[i])
        retiros_lista_pag_1.append(tabla_pag_1[i+1])
        
retiros_lista_pag_1

['RETIROS 8,000.00', 'DEPOSITOS']

In [60]:
retiros1=retiros_lista_pag_1[0].split('RETIROS ')
retiros1_1=[]
for i in retiros1:
    if re.search(r'\d+', i):
        retiros1_1.append(i)

In [153]:
#organizo retiros pagina 1
retiros_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('RETIRO'):
        retiros_pag_1.append(retiros1_1[contador])
        contador=contador+1
    else:
        retiros_pag_1.append('0')
retiros_pag_1

['0', '0', '0', '8,000.00', '0']

In [160]:
#depositos
depositos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('depositos',tabla_pag_1[i].lower()):
        depositos_lista_pag_1.append(tabla_pag_1[i])
        depositos_lista_pag_1.append(tabla_pag_1[i+1])
        depositos_lista_pag_1.append(tabla_pag_1[i+2])
        
depositos_lista_pag_1

['DEPOSITOS', '209.13 209.13', '186.34']

In [93]:
depositos1=[]
for i in depositos_lista_pag_1:
    if re.search(r'\d+', i):
        depositos1.append(i)

depositos1_1=[]     
for i in depositos1:
    j=i.split(' ')
    for k in j:
        depositos1_1.append(k)

In [94]:
depositos1_1

['209.13', '209.13', '186.34']

In [121]:
#organizo depositos
depositos_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('DEPOSITO') or conceptos_pag_1[i].startswith('INTERES'):
        depositos_pag_1.append(depositos1_1[contador])
        contador=contador+1
    else:
        depositos_pag_1.append('0')
depositos_pag_1

['0', '209.13', '209.13', '0', '186.34']

In [98]:
#saldos
saldos_lista_pag_1=[]
for i in range(len(tabla_pag_1)):
    if re.match('saldo',tabla_pag_1[i].lower()):
        saldos_lista_pag_1.append(tabla_pag_1[i])
        saldos_lista_pag_1.append(tabla_pag_1[i+1])
        saldos_lista_pag_1.append(tabla_pag_1[i+2])
        
saldos_lista_pag_1

['SALDO ANTERIOR INTERESES AL 1.27% INTERESES AL 1.27% RETIRO 72825 INTERESES AL 1.23%',
 'RETIROS 8,000.00',
 'DEPOSITOS',
 'SALDO 10,366.75 10,575.88 10,785.01',
 '2,971.35',
 '000182.B10OTDA006.OD.0930.01']

In [105]:
saldos1_1=[]
for i in range(len(saldos_lista_pag_1)):
    if re.search(r'SALDO \d+', saldos_lista_pag_1[i]):
        saldos1_1.append(saldos_lista_pag_1[i])
        saldos1_1.append(saldos_lista_pag_1[i+1])
        
saldos1_1

['SALDO 10,366.75 10,575.88 10,785.01', '2,971.35']

In [113]:
saldos1_2=[]
for i in saldos1_1:
    j=i.split(' ')
    saldos1_2.append(j)
    
saldos1_3=[]
for i in saldos1_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos1_3.append(j)
            
saldos1_3

['10,366.75', '10,575.88', '10,785.01', '2,971.35']

In [120]:
#organizo saldos
saldos_pag_1=[]
contador=0
for i in range(len(conceptos_pag_1)):
    if conceptos_pag_1[i].startswith('SALDO') or conceptos_pag_1[i].startswith('INTERES') :
        saldos_pag_1.append(saldos1_3[contador])
        contador=contador+1
    else:
        saldos_pag_1.append('0')
saldos_pag_1

['10,366.75', '10,575.88', '10,785.01', '0', '2,971.35']

# pagina 2

In [122]:
tabla_pag_2= pdf[indices_detalle[1]:-1]
tabla_pag_2

['DETALLE DE OPERACIONES',
 'FECHA',
 '04 SEP 05 SEP 06 SEP 07 SEP 08 SEP 09 SEP 10 SEP 10 SEP 11 SEP 11 SEP 12 SEP 13 SEP 14 SEP 14 SEP 15 SEP 16 SEP 17 SEP 17 SEP 18 SEP 19 SEP 20 SEP 21 SEP 22 SEP 23 SEP 24 SEP 24 SEP 25 SEP 26 SEP 27 SEP 28 SEP 29 SEP 30 SEP',
 'CONCEPTO',
 'INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% DEPOSITO 113148 INTERESES AL 1.22% DEPOSITO 139775 INTERESES AL 1.62% INTERESES AL 1.62% INTERESES AL 1.62% RETIRO 334585 INTERESES AL 1.44% INTERESES AL 1.44% INTERESES AL 1.44% RETIRO 189211 INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% RETIRO 64331 INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91%',
 'RETIROS',
 '8,000.00 7,000.00',
 '465,000.00',
 'DEPOSITOS 186.35 186.36 186.36 186.37 186.38 186.38',
 '2,000.00 188.93',
 '7,000

In [124]:
#fecha
fecha_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('fecha',tabla_pag_2[i].lower()):
        fecha_lista_pag_2.append(tabla_pag_2[i+1])
        
fecha_lista_pag_2

['04 SEP 05 SEP 06 SEP 07 SEP 08 SEP 09 SEP 10 SEP 10 SEP 11 SEP 11 SEP 12 SEP 13 SEP 14 SEP 14 SEP 15 SEP 16 SEP 17 SEP 17 SEP 18 SEP 19 SEP 20 SEP 21 SEP 22 SEP 23 SEP 24 SEP 24 SEP 25 SEP 26 SEP 27 SEP 28 SEP 29 SEP 30 SEP']

In [126]:
#separo
fechas_pag_2=re.findall(r'\d{2} \w{3}',fecha_lista_pag_2[0])

In [129]:
len(fechas_pag_2)

32

In [128]:
#conceptos
conceptos_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('concepto',tabla_pag_2[i].lower()):
        conceptos_lista_pag_2.append(tabla_pag_2[i+1])
        
conceptos_lista_pag_2

['INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% INTERESES AL 1.23% DEPOSITO 113148 INTERESES AL 1.22% DEPOSITO 139775 INTERESES AL 1.62% INTERESES AL 1.62% INTERESES AL 1.62% RETIRO 334585 INTERESES AL 1.44% INTERESES AL 1.44% INTERESES AL 1.44% RETIRO 189211 INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% INTERESES AL 1.24% RETIRO 64331 INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91% INTERESES AL 0.91%',
 'Descripción Servicios de Facturación Valor Unitario']

In [133]:
conceptos_prueba2=re.split('SALDO |INTERESES | RET| DE',conceptos_lista_pag_2[0])
conceptos_prueba2

['',
 'AL 1.23% ',
 'AL 1.23% ',
 'AL 1.23% ',
 'AL 1.23% ',
 'AL 1.23% ',
 'AL 1.23%',
 'POSITO 113148 ',
 'AL 1.22%',
 'POSITO 139775 ',
 'AL 1.62% ',
 'AL 1.62% ',
 'AL 1.62%',
 'IRO 334585 ',
 'AL 1.44% ',
 'AL 1.44% ',
 'AL 1.44%',
 'IRO 189211 ',
 'AL 1.24% ',
 'AL 1.24% ',
 'AL 1.24% ',
 'AL 1.24% ',
 'AL 1.24% ',
 'AL 1.24% ',
 'AL 1.24%',
 'IRO 64331 ',
 'AL 0.91% ',
 'AL 0.91% ',
 'AL 0.91% ',
 'AL 0.91% ',
 'AL 0.91% ',
 'AL 0.91% ',
 'AL 0.91%']

In [135]:
conceptos_pag_2=[]
for i in range(len(conceptos_prueba2)):
    if conceptos_prueba2[i].startswith('ANTERIOR'):
        j='SALDO '+conceptos_prueba2[i]
        conceptos_pag_2.append(j)
    elif conceptos_prueba2[i].startswith('AL'):
        j='INTERES '+conceptos_prueba2[i]
        conceptos_pag_2.append(j)
    elif conceptos_prueba2[i].startswith('IRO'):
        j='RET'+ conceptos_prueba2[i]
        conceptos_pag_2.append(j)
    elif conceptos_prueba2[i].startswith('POSITO'):
        j='DE'+ conceptos_prueba2[i]
        conceptos_pag_2.append(j)

conceptos_pag_2

['INTERES AL 1.23% ',
 'INTERES AL 1.23% ',
 'INTERES AL 1.23% ',
 'INTERES AL 1.23% ',
 'INTERES AL 1.23% ',
 'INTERES AL 1.23%',
 'DEPOSITO 113148 ',
 'INTERES AL 1.22%',
 'DEPOSITO 139775 ',
 'INTERES AL 1.62% ',
 'INTERES AL 1.62% ',
 'INTERES AL 1.62%',
 'RETIRO 334585 ',
 'INTERES AL 1.44% ',
 'INTERES AL 1.44% ',
 'INTERES AL 1.44%',
 'RETIRO 189211 ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24% ',
 'INTERES AL 1.24%',
 'RETIRO 64331 ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91% ',
 'INTERES AL 0.91%']

In [141]:
#retiros
retiros_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('retiros',tabla_pag_2[i].lower()):
        retiros_lista_pag_2.append(tabla_pag_2[i])
        retiros_lista_pag_2.append(tabla_pag_2[i+1])
        retiros_lista_pag_2.append(tabla_pag_2[i+2])
        
retiros_lista_pag_2

['RETIROS', '8,000.00 7,000.00', '465,000.00']

In [ ]:
retiros1=retiros_lista_pag_1[0].split('RETIROS ')
retiros1_1=[]
for i in retiros1:
    if re.search(r'\d+', i):
        retiros1_1.append(i)

In [149]:
retiros2_1=[]
for i in retiros_lista_pag_2:
    retiros2_1.append(i.split(' '))
    
retiros2_1

[['RETIROS'], ['8,000.00', '7,000.00'], ['465,000.00']]

In [150]:
retiros2_2=[]
for i in retiros2_1:
    for j in i:
        if re.search(r'\d+', j):
            retiros2_2.append(j)
        
retiros2_2

['8,000.00', '7,000.00', '465,000.00']

In [152]:
#organizo retiros pagina 2
retiros_pag_2=[]
contador=0
for i in range(len(conceptos_pag_2)):
    if conceptos_pag_2[i].startswith('RETIRO'):
        retiros_pag_2.append(retiros2_2[contador])
        contador=contador+1
    else:
        retiros_pag_2.append('0')
retiros_pag_2

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '8,000.00',
 '0',
 '0',
 '0',
 '7,000.00',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '465,000.00',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0']

In [166]:
#depositos
depositos_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('depositos',tabla_pag_2[i].lower()):
        depositos_lista_pag_2.append(tabla_pag_2[i])
        depositos_lista_pag_2.append(tabla_pag_2[i+1])
        depositos_lista_pag_2.append(tabla_pag_2[i+2])
        depositos_lista_pag_2.append(tabla_pag_2[i+3])
        depositos_lista_pag_2.append(tabla_pag_2[i+4])
        depositos_lista_pag_2.append(tabla_pag_2[i+5])
        
depositos_lista_pag_2

['DEPOSITOS 186.35 186.36 186.36 186.37 186.38 186.38',
 '2,000.00 188.93',
 '7,000.00 507.15 507.18 507.20',
 '330.73 330.75 330.76',
 '178.65 178.65 178.66 178.66 178.67 178.68 178.68',
 '119.15 119.15 119.16 119.15 119.16 119.17 119.17']

In [175]:
depositos2_1=[]
for i in depositos_lista_pag_2:
    j=i.split('DEPOSITOS')
    depositos2_1.append(j)

In [176]:
depositos2_2=[]
for i in depositos2_1:
    for j in i:
        if re.search(r'\d+', j):
            depositos2_2.append(j.strip())
        
depositos2_2

['186.35 186.36 186.36 186.37 186.38 186.38',
 '2,000.00 188.93',
 '7,000.00 507.15 507.18 507.20',
 '330.73 330.75 330.76',
 '178.65 178.65 178.66 178.66 178.67 178.68 178.68',
 '119.15 119.15 119.16 119.15 119.16 119.17 119.17']

In [178]:
depositos2_3=[]
for i in depositos2_2:
    j=i.split(' ')
    for k in j:
        depositos2_3.append(k)

In [180]:
#organizo depositos
depositos_pag_2=[]
contador=0
for i in range(len(conceptos_pag_2)):
    if conceptos_pag_2[i].startswith('DEPOSITO') or conceptos_pag_2[i].startswith('INTERES'):
        depositos_pag_2.append(depositos2_3[contador])
        contador=contador+1
    else:
        depositos_pag_2.append('0')
len(depositos_pag_2)

32

In [193]:
#saldos
saldos_lista_pag_2=[]
for i in range(len(tabla_pag_2)):
    if re.match('saldo',tabla_pag_2[i].lower()):
        saldos_lista_pag_2.append(tabla_pag_2[i])
        saldos_lista_pag_2.append(tabla_pag_2[i+1])
        saldos_lista_pag_2.append(tabla_pag_2[i+2])
        saldos_lista_pag_2.append(tabla_pag_2[i+3])
        saldos_lista_pag_2.append(tabla_pag_2[i+4])
        saldos_lista_pag_2.append(tabla_pag_2[i+5])
        
saldos2_1=saldos_lista_pag_2

In [196]:
saldos2_2=[]
for i in saldos2_1:
    j=i.split(' ')
    saldos2_2.append(j)
    
saldos2_3=[]
for i in saldos2_2:
    for j in i:
        if re.search(r'\d+',j):
            saldos2_3.append(j)

In [198]:
#organizo saldos
saldos_pag_2=[]
contador=0
for i in range(len(conceptos_pag_2)):
    if conceptos_pag_2[i].startswith('SALDO') or conceptos_pag_2[i].startswith('INTERES') :
        saldos_pag_2.append(saldos2_3[contador])
        contador=contador+1
    else:
        saldos_pag_2.append('0')


32

In [201]:
#junto toda la información
fechas= fechas_pag_1+fechas_pag_2
conceptos = conceptos_pag_1+conceptos_pag_2
retiros= retiros_pag_1+retiros_pag_2
depositos= depositos_pag_1+depositos_pag_2
saldos= saldos_pag_1+saldos_pag_2

In [202]:
#este data frame se puede exportar como csv
edo_cuenta=pd.DataFrame(list(zip(fechas, conceptos, retiros, depositos, saldos)),
              columns=['fecha','concepto','retiro', 'deposito','saldo'])

In [203]:
edo_cuenta

,fecha,concepto,retiro,deposito,saldo
0,01 SEP,SALDO ANTERIOR,0,0,"10,366.75"
1,01 SEP,INTERES AL 1.27%,0,209.13,"10,575.88"
2,02 SEP,INTERES AL 1.27%,0,209.13,"10,785.01"
3,03 SEP,RETIRO 72825,"8,000.00",0,0
4,03 SEP,INTERES AL 1.23%,0,186.34,"2,971.35"
5,04 SEP,INTERES AL 1.23%,0,186.35,"3,157.70"
6,05 SEP,INTERES AL 1.23%,0,186.36,"3,344.06"
7,06 SEP,INTERES AL 1.23%,0,186.36,"3,530.42"
8,07 SEP,INTERES AL 1.23%,0,186.37,"3,716.79"
9,08 SEP,INTERES AL 1.23%,0,186.38,"3,903.17"
